# Verification
---
Ici on va vérifier que le message n'a pas été altéré en recalculant sa signature et en la comparant avec celle reçue

## Dechiffrement du message signé
---

In [8]:
def decipherRSA(ciphertext: int, n: int, key: int) -> str:
    """
    Decrypts an RSA ciphertext using the private key.

    Args:
        ciphertext (int): The encrypted message.
        n (int): The modulus.
        key (int): The decryption key (private exponent).

    Returns:
        str: The original plaintext message.
    """
    # Extract individual digits from the ciphertext
    msg_cipher = []
    while ciphertext != 0:
        ciphertext, remainder = divmod(ciphertext, n)
        msg_cipher.append(remainder)

    # Decrypt each block using modular exponentiation
    msg_block = [str(pow(i, key, n)) for i in msg_cipher]

    # Convert blocks back to characters
    msg_padded = []
    for block in msg_block:
        block_padded = []
        for i in range(len(block), 3, -3):
            block_padded.append(chr(int(block[i - 3 : i])))
        block_padded.append(chr(int(block[: i - 3])))
        msg_padded.extend(reversed(block_padded))

    return "".join(msg_padded)

# Loading of the signed message and of the public key

with open("Signature/SignedMessage.txt", "r") as f:
    signedMessage = int(f.read())

with open("Signature/PublicKey.txt", "r") as f:
    n = int(f.readline())
    e = int(f.readline())

print("Message signé = \n", signedMessage)

ReceivedhashMessage = decipherRSA(signedMessage, n, e)

print("\nMessage haché reçu = \n", ReceivedhashMessage)

Message signé = 
 1423486093834684304923184388447221676132257136666673732674072102525156824882584855051924977976045805227320852295345029291455342210776534589284646700064165156211895469234879876117354283626910174335602653112229250820025844134935059678045649175312666123822575369802034873953880299472453999154092609418394180509051126053496527225864174938470674957586109440003641204524669068609735316493398463172164216451279367698537745101443823534749914047740322295701280169713958781759130250054964576174703931401470234865851211119357929847625180699058001988360015809266654542571181597285992504042084684621432107733772172277231435351197

Message haché reçu = 
 e441e17cdbaccb456039f7cf142c0cf3


## Rehachage du message
---

In [9]:
import math

# MD5 Implementation

def leftRotate(value: int, shift: int) -> int:
    """Performs a left rotation (circular shift) on a 32-bit integer."""
    return ((value << shift) | (value >> (32 - shift))) & 0xFFFFFFFF

def MD5(message: str) -> str:
    """
    Calculates the MD5 hash of the input message.

    Args:
        message (str): The input message to hash.

    Returns:
        str: The hexadecimal representation of the MD5 hash.
    """
    # Initialize constants
    K = [int(abs(math.sin(i + 1)) * 2 ** 32) for i in range(64)]
    s = [7, 12, 17, 22] * 4 + [5, 9, 14, 20] * 4 + [4, 11, 16, 23] * 4 + [6, 10, 15, 21] * 4 # shifts values

    # Convert the message to bytes
    messageBytes = message.encode('utf-8')

    # Initialize state variables
    a, b, c, d = 0x67452301, 0xefcdab89, 0x98badcfe, 0x10325476

    # Padding the message
    messageLength = len(messageBytes) * 8
    messageBytes += b'\x80'
    
    # In fact len(messageBytes) * 8 must be equal to 56 * 8 (448) mod 64* 8 = 512
    while len(messageBytes) % 64 != 56: 
        messageBytes += b'\x00'
    messageBytes += messageLength.to_bytes(8, byteorder='little')

    # Process the message in Blocks of 64 bytes
    for i in range(0, len(messageBytes), 64):
        Block = messageBytes[i:i+64]
        SubBlocks = [int.from_bytes(Block[j:j+4], byteorder='little') for j in range(0, 64, 4)]

        A, B, C, D = a, b, c, d

        for j in range(64):
            if j <= 15:
                F_func = (B & C) | (~B & D)
                g = j
            elif j <= 31:
                F_func = (D & B) | (~D & C)
                g = (5 * j + 1) % 16
            elif j <= 47:
                F_func = B ^ C ^ D
                g = (3 * j + 5) % 16
            else:
                F_func = C ^ (B | ~D)
                g = (7 * j) % 16

            temp = D
            D = C
            C = B
            B = (B + leftRotate(A + F_func + K[j] + SubBlocks[g], s[j])) & 0xFFFFFFFF
            A = temp

        a = (a + A) & 0xFFFFFFFF
        b = (b + B) & 0xFFFFFFFF
        c = (c + C) & 0xFFFFFFFF
        d = (d + D) & 0xFFFFFFFF

    # Concatenate the final state variables to obtain the MD5 hash
    hashMD5 = (a.to_bytes(4, byteorder='little') +
                 b.to_bytes(4, byteorder='little') +
                 c.to_bytes(4, byteorder='little') +
                 d.to_bytes(4, byteorder='little'))

    # Convert the MD5 hash to hexadecimal representation
    hashMD5Hex = ''.join(format(byte, '02x') for byte in hashMD5)

    return hashMD5Hex

# Opening of the message to be signed

with open("Signature/Message.txt", "r") as f:
    message = f.read()

print("Message reçu = \n", message)

# Message hashing

hashMessage = MD5(message)

print("\nMessage haché = \n", hashMessage)

Message reçu = 
 Salut! On fait un test de notre implÃ©mentation des signatures numÃ©riques avec MD5 et RSA!
On va signer ce message et par la suite vÃ©rifier s'il n'a pas Ã©tÃ© altÃ©rÃ©!
A plus pour la vÃ©rification...

Message haché = 
 e441e17cdbaccb456039f7cf142c0cf3


## Verification
---

In [10]:
print("Message haché reçu = \n", ReceivedhashMessage)

print("\nMessage haché = \n", hashMessage)

if ReceivedhashMessage == hashMessage:
    print("\nVérification réussie! Le message n'a  pas été altéré. Vous pouvez le lire en toute quiétude!")
else:
    print("\nEchec de la vérification! Le message a été altéré! Veuillez démander qu'on vous le renvoie!")

Message haché reçu = 
 e441e17cdbaccb456039f7cf142c0cf3

Message haché = 
 e441e17cdbaccb456039f7cf142c0cf3

Vérification réussie! Le message n'a  pas été altéré. Vous pouvez le lire en toute quiétude!
